In [1]:
#@title MIT License
#
# Copyright (c) 2019 Andreas Eberlein
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Prepare data for training a deep neural network for semantic segmentation based on Virtual KITTI dataset

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt

import cv2

In [3]:
import data_helper

In [4]:
rgb_image_folder = "../Data_compressed/vkitti_1.3.1_rgb"
seg_image_folder = "../Data_compressed/vkitti_1.3.1_scenegt"
encoding_file_path = "../Data_compressed/vkitti_1.3.1_scenegt/0001_sunset_scenegt_rgb_encoding.txt"

Get lists of training and validation files and read them from hard drive:

In [5]:
train_image_folder = os.path.join(rgb_image_folder, '0001/sunset')
train_image_files = os.listdir(train_image_folder)
print("Number of training images for now: ", len(train_image_files))

# valid_image_folder = os.path.join(seg_image_folder, '0002/sunset')
# valid_image_files = os.listdir(valid_image_folder)
# print("Number of validation images for now: ", len(valid_image_files))

Number of training images for now:  447


In [6]:
train_label_folder = os.path.join(seg_image_folder, '0001/sunset')
train_label_files = os.listdir(train_label_folder)
print("Number of training label images for now: ", len(train_label_files))

# valid_label_folder = os.path.join(seg_image_folder, '0002/sunset')
# valid_label_files = os.listdir(valid_label_folder)
# print("Number of validation images for now: ", len(valid_label_files))

Number of training label images for now:  447


In [7]:
train_images = data_helper.loadImagesToArray(train_image_folder, train_image_files)
# valid_images = data_helper.loadImagesToArray(valid_image_folder, valid_image_files)

train_labels = data_helper.loadImagesToArray(train_label_folder, train_label_files)
# valid_labels = data_helper.loadImagesToArray(valid_label_folder, valid_label_files)

Next step: Reading encoding data and convert labels to one-hot encoded array

In [17]:
instance_dict = data_helper.parseCategoryFile(encoding_file_path)
values_categories_dict, categories_ids_dict, ids_categories_dict, ids_values_dict = \
        data_helper.reduceInstancesToCategories(instance_dict)
values_ids_dict = data_helper.oneHotEncodingDict(values_categories_dict, categories_ids_dict)

print(categories_ids_dict)

{'GuardRail': 0, 'Terrain': 1, 'Car': 2, 'Pole': 3, 'Vegetation': 4, 'Truck': 5, 'TrafficSign': 6, 'Sky': 7, 'TrafficLight': 8, 'Misc': 9, 'Van': 10, 'Road': 11, 'Building': 12, 'Tree': 13}


In [9]:
train_labels.shape

(447, 125, 414, 3)

In [10]:
one_hot_encoded_labels = data_helper.oneHotEncodeImages(train_labels, values_ids_dict)

Note: Everytime we recompute the dictionaries that map values to categories, the order of the elements in the dictionary may change. Therefore and in order to simplify further processing, we store the data in an hdf5 file.

TODO: We should sort the dictionaries according to the name of the categories. This would allow to establish a stable sorting!

In [11]:
import h5py
processed_training_data_file_name = "../Data_compressed/Preprocessed/ProcessedTrainingData.hdf5"

data_file = h5py.File(processed_training_data_file_name, 'w')
labels_data_set = data_file.create_dataset('one_hot_encoded_labels', one_hot_encoded_labels.shape, 
                        dtype=one_hot_encoded_labels.dtype)
labels_data_set[...] = one_hot_encoded_labels

images_data_set = data_file.create_dataset('training_images', train_images.shape,
                                   dtype=train_images.dtype)
images_data_set[...] = train_images

data_file.close()

In [12]:
import pickle
dict_file_name = '../Data_compressed/Preprocessed/Dictionaries.dat'
dict_file = open(dict_file_name, 'wb')
value_category_id_mappings = [values_categories_dict, categories_ids_dict, 
                              ids_categories_dict, ids_values_dict, values_ids_dict]
pickle.dump(value_category_id_mappings, dict_file, pickle.HIGHEST_PROTOCOL)

dict_file.close()

## To do:
- Enlarge dataset by loading more folders and merging of labels files
- Normalize input images from pixel values of 0...255 to 0...1 (and float values)